In [29]:
! pip install python-dotenv --quiet
! pip install azure-search-documents==11.6.0b4 --quiet
! pip install azure-identity --quiet
! pip install openai --quiet


In [30]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
#endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
endpoint = "https://kodeco-ai-search-service-paid-basic.search.windows.net"
#credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()
credential = AzureKeyCredential("KdBoplAH20sep1dayhZDh34Uvg2K7S3UyTzqRyutD9AzSeDMLPmE")
#index_name = os.getenv("AZURE_SEARCH_INDEX", "vectest")
index_name = "vectest"
#azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_endpoint = "https://kodeco-ai-search-paid-service.openai.azure.com/"
#azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "") if len(os.getenv("AZURE_OPENAI_KEY", "")) > 0 else None
azure_openai_key = "985e7ef97c6e403f8784927d1354c8a1"
#azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_deployment = "text-embedding-3-large-kodeco-ai-new"
#azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1024))
azure_openai_embedding_dimensions = 1024
#embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
embedding_model_name = "text-embedding-3-large"
#azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")
azure_openai_api_version = "2024-06-01"

In [31]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_deployment=azure_openai_embedding_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)

# Generate Document Embeddings using OpenAI Ada 002
# Read the text-sample.json
path = os.path.join('text-sample-2.json')
with open(path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

titles = [item['title'] for item in input_data]
content = [item['content'] for item in input_data]
title_response = client.embeddings.create(input=titles, model=embedding_model_name, dimensions=azure_openai_embedding_dimensions)
title_embeddings = [item.embedding for item in title_response.data]
content_response = client.embeddings.create(input=content, model=embedding_model_name, dimensions=azure_openai_embedding_dimensions)
content_embeddings = [item.embedding for item in content_response.data]

# Generate embeddings for title and content fields
for i, item in enumerate(input_data):
    title = item['title']
    content = item['content']
    item['titleVector'] = title_embeddings[i]
    item['contentVector'] = content_embeddings[i]

# Output embeddings to docVectors.json file
output_path = os.path.join('docVectors.json')
# output_directory = os.path.dirname(output_path)
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)
with open(output_path, "w") as f:
    json.dump(input_data, f)

In [32]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)


# Create a search index
index_client = SearchIndexClient(
    endpoint=endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=azure_openai_embedding_dimensions, vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=azure_openai_embedding_dimensions, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer="myVectorizer"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myVectorizer",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=azure_openai_endpoint,
                deployment_id=azure_openai_embedding_deployment,
                model_name=embedding_model_name,
                api_key=azure_openai_key
            )
        )
    ]
)



semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


 vectest created


In [33]:
from azure.search.documents import SearchClient
import json

# Upload some documents to the index
output_path = os.path.join('docVectors.json')
# output_directory = os.path.dirname(output_path)
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)
with open(output_path, 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 50 documents


In [34]:
# from azure.search.documents.models import VectorizedQuery

# # Pure Vector Search
# query = "intergalactic"  
# embedding = client.embeddings.create(input=query, model=embedding_model_name, dimensions=azure_openai_embedding_dimensions).data[0].embedding

# vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="contentVector")
  
# results = search_client.search(  
#     search_text=None,  
#     vector_queries= [vector_query],
#     select=["title", "content", "category"],
# )  
  
# for result in results:  
#     print(f"Title: {result['title']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Content: {result['content']}")  
#     print(f"Category: {result['category']}\n")  


In [35]:
from azure.search.documents.models import VectorizableTextQuery

# Hybrid Search
query = "funny entertainment"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Parks and Recreation
Score: 0.01666666753590107
Content: Parks and Recreation is a mockumentary-style comedy that follows the employees of the Parks and Recreation Department in the fictional town of Pawnee, Indiana. The series is centered around Leslie Knope, an enthusiastic and optimistic public servant, and her quirky coworkers. The show's humor is rooted in its loveable characters, witty dialogue, and its satirical take on small-town government. Parks and Recreation has gained a dedicated fanbase for its heartwarming moments, memorable quotes, and its positive portrayal of community and public service.
Category: Comedy

Title: The Good Place
Score: 0.016393441706895828
Content: The Good Place is a unique and clever comedy that explores the concept of the afterlife with humor and philosophical depth. The series follows Eleanor Shellstrop, a woman who finds herself in the Good Place, a utopian afterlife, despite not living a particularly virtuous life. As Eleanor tries to beco

In [36]:
# Hybrid Search
query = "funny entertainment"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", weight=0.2)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n") 

Title: Parks and Recreation
Score: 0.0033333336468786
Content: Parks and Recreation is a mockumentary-style comedy that follows the employees of the Parks and Recreation Department in the fictional town of Pawnee, Indiana. The series is centered around Leslie Knope, an enthusiastic and optimistic public servant, and her quirky coworkers. The show's humor is rooted in its loveable characters, witty dialogue, and its satirical take on small-town government. Parks and Recreation has gained a dedicated fanbase for its heartwarming moments, memorable quotes, and its positive portrayal of community and public service.
Category: Comedy

Title: The Good Place
Score: 0.003278688294813037
Content: The Good Place is a unique and clever comedy that explores the concept of the afterlife with humor and philosophical depth. The series follows Eleanor Shellstrop, a woman who finds herself in the Good Place, a utopian afterlife, despite not living a particularly virtuous life. As Eleanor tries to becom

In [37]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "what is friends?"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", exhaustive=True)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: Friends. Comedy. Friends is<em> a beloved sitcom that follows the lives of six close-knit friends—Rachel, Ross, Monica, Chandler, Joey, and Phoebe—as they navigate</em> the ups and downs of life in New York City. With its iconic catchphrases, humorous scenarios, and relatable characters, Friends has become a cultural touchstone, resonating with audiences around the world. The show balances humor with heart, exploring themes of love, friendship, and the challenges of adulthood. Its enduring popularity is a testament to its timeless appeal and the chemistry between its cast..
Semantic Answer Score: 0.9609375

Title: Friends
Reranker Score: 3.632352828979492
Content: Friends is a beloved sitcom that follows the lives of six close-knit friends—Rachel, Ross, Monica, Chandler, Joey, and Phoebe—as they navigate the ups and downs of life in New York City. With its iconic catchphrases, humorous scenarios, and relatable characters, Friends has become a cultural touchstone, reson

In [38]:
#index_client.delete_index(index)


: 